In [14]:
## Import all the things. 
import numpy as np
import csv
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from nltk.corpus import opinion_lexicon as op
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import pos_tag
np.set_printoptions(linewidth=400)   # optional: widens column of numpy array display

In [15]:
## get positve and negative words from the lexicon. 
pos_words = set(op.words('positive-words.txt'))
neg_words = set(op.words('negative-words.txt'))

In [16]:
## create lists for response and target. 
X = [] ; y = []; initial_set = []; X_test = []; y_test = []
with open("reviews.txt", 'r', encoding='utf8') as reviews:
    for review in reviews:
        if review != "":
            review = review.strip()
            review = review.lower()
            #review = word_tokenize(review)
            if len(review) > 1:
                initial_set.append(review)
    random.shuffle(initial_set)
num_sentences = len(initial_set)

In [17]:
# write out a split to files:
train_len = int(0.8 * num_sentences)
test_len = num_sentences - train_len

train_list = initial_set[:train_len]
test_list = initial_set[train_len:]
print(len(train_list))
print(len(test_list))

with open('reviews-train.txt', 'w') as r_train:
    for line in train_list:
        line = str(line)
        r_train.write(line)
        r_train.write('\n')
with open('reviews-test.txt', 'w') as r_test:
    for test_line in test_list:
        test_line = str(test_line)
        r_test.write(test_line)
        r_test.write('\n')

1999
500


In [18]:
def return_rating(pos, neg):
        stars = [0,0,0,0,0]
        if negCount > (posCount*2):
            stars[0] = 1
        elif posCount > (negCount*2):
            stars[4] = 1
        elif negCount == posCount:
            stars[2] = 1
        elif negCount < posCount:
            stars[3]= 1
        elif posCount < negCount:
            stars[1] = 1
            

In [19]:
#rev_train_file = open('reviews-train.txt')
rev_train_file = open('reviews-train.txt')

lines = rev_train_file.readlines()
for line in lines:
    line = line.strip()
    X.append(word_tokenize(line[:-1]))
    y.append(line[-1])
    
rev_train_file.close()

In [20]:
#rev_train_file = open('reviews-train.txt')
rev_test_file = open('reviews-test.txt')

lines = rev_test_file.readlines()
for line in lines:
    line = line.strip()
    X_test.append(word_tokenize(line[:-1]))
    y_test.append(line[-1])
    
####rev_test_file.close()

In [24]:
###################################################################################################################
############################################ FEATURE EXTRACTION ###################################################
###################################################################################################################
def np_feature_extraction(X,y):
    X_list = []
    y_list = []
    my_dict = {}
    
    for x,line in enumerate(X):
        rating = int(y[x])
        posCount = 0 ; negCount = 0
        # create features of positive and negative words. 
        leng = len(line)
        for word in line:
            if word in pos_words:
                posCount +=1
            elif word in neg_words:
                negCount+=1
        ## Check number of ratings. 
        my_dict[rating]=my_dict.get(rating,0)+1
        
        line = [leng, posCount, negCount]
        X_list.append(line)
        y_list.append(rating)
    
    print(my_dict)
    
    X_np = (np.array(X_list))
    y_np = (np.array(y_list))
    
    return X_np,y_np
#########################################################################################################################

In [25]:
X_train_np, y_train_np = np_feature_extraction(X,y)
X_test_np, y_test_np = np_feature_extraction(X_test, y_test)


{4: 393, 5: 1367, 3: 147, 2: 56, 1: 36}
{3: 25, 5: 342, 4: 101, 1: 14, 2: 18}


In [23]:
for mclass in ('multinomial', 'ovr'):
    lr = LogisticRegression(solver='lbfgs', max_iter=3000, random_state=0, multi_class=mclass).fit(X_train_np, y_train_np)
    yhat = lr.predict(X_test_np)
     
    # the 3 lines below show how to invoke various output    
    print("\n",mclass,"Accuracy",accuracy_score(y_test_np, yhat))
    print("\n",mclass,"Classification Report\n",classification_report(y_test_np, yhat),sep="")
    print("\n",mclass,"Classification Report\n",confusion_matrix(y_test_np, yhat),sep="")


 multinomial Accuracy 1.0

multinomialClassification Report
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00        18
           3       1.00      1.00      1.00        25
           4       1.00      1.00      1.00       101
           5       1.00      1.00      1.00       342

   micro avg       1.00      1.00      1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500


multinomialClassification Report
[[ 14   0   0   0   0]
 [  0  18   0   0   0]
 [  0   0  25   0   0]
 [  0   0   0 101   0]
 [  0   0   0   0 342]]

 ovr Accuracy 0.91

ovrClassification Report
              precision    recall  f1-score   support

           1       1.00      0.86      0.92        14
           2       0.00      0.00      0.00        18
           3       0.00      0.00      0.00        25
           4       0.71      1.00    

/home/loki/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/loki/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
